In [1]:
import numpy as np
wlist=np.load('rateListUsersYahoo.npy')[0]/max(np.load('rateListUsersYahoo.npy')[0])/4

In [2]:
def hwithoutconstraints(ranked_action):
    res=[0]*len(wlist)
    gamma=0.9
    currentw=wlist[ranked_action]
    p=0
    for i in range(len(ranked_action)):
        if i==0:
            res[i]=np.random.choice([0,1],p=[1-wlist[ranked_action[0]],wlist[ranked_action[0]]])
            if res[i]==1:
                return res
        else:
            if np.random.choice([0,1],p=[1-gamma,gamma])==0:
                return res
            else:
                res[i]=np.random.choice([0,1],p=[1-wlist[ranked_action[i]],wlist[ranked_action[i]]])
                if res[i]==1:
                    return res
    return res

In [3]:

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import gurobipy
import numpy as np
import argparse
from copy import deepcopy
import torch
import gym
import time
import torch
import torch.nn as nn	
import torch.nn.functional as F	
import torch.optim as optim	
import numpy as np

import argparse
import sys


import numpy as np
from numpy.random import permutation
import matplotlib.pyplot as plt

class para:
    def __init__(self):
        self.batch_size=16
        self.wmc=10
        self.solDim=24
        self.card=5
        self.test_batch_size=1000
        self.epochs=10
        self.lr=0.1
        self.momentum=0.5
        self.no_cuda='store_true'
        self.seed=1
        self.log_interval=10
        self.hidden_dim=10
args=para()
args5=para()
legalList={0: [13, 18],
 1: [17, 20, 22],
 2: [4, 5, 10, 13, 14, 16, 17, 18, 20, 22],
 3: [5, 9, 11, 12, 15],
 4: [2, 10, 13, 14, 16, 17, 18, 20, 22],
 5: [2, 3, 6, 9, 10, 11, 12, 14, 16],
 6: [5, 15],
 7: [],
 8: [],
 9: [3, 5, 11, 12, 15],
 10: [2, 4, 5, 14, 16, 22],
 11: [3, 5, 9, 12, 15],
 12: [3, 5, 9, 11, 15],
 13: [0, 2, 4, 14, 18, 22],
 14: [2, 4, 5, 10, 13, 16, 17, 18, 20, 22],
 15: [3, 6, 9, 11, 12, 17, 19, 20], 
 16: [2, 4, 5, 10, 14, 22],
 17: [1, 2, 4, 14, 15, 19, 20, 22],
 18: [0, 2, 4, 13, 14, 22],
 19: [15, 17, 20],
 20: [1, 2, 4, 14, 15, 17, 19, 22],
 21: [],
 22: [1, 2, 4, 10, 13, 14, 16, 17, 18, 20],
 23: []}


def checkFea(x):
    cnt=0
    cntT=0
    for i in range(len(legalList.keys())):
        for j in legalList[i]:
            cntT+=1
            if x[j]+x[i]>1:
                cnt+=1
    return cnt/cntT/2
def checkFea2(x):
    cnt=0
    cntT=0
    for i in range(len(legalList.keys())):
        for j in legalList[i]:
            cntT+=1
            if x[j]+x[i]>1:
                cnt+=1
    return cnt/cntT
def CB(alpha,x,M):
    return alpha*np.sqrt(np.dot(np.dot(x.T,np.linalg.inv(M)),x))
def solver(theta,MODEL):

    import gurobipy
    import time
    s=time.time()
    
    variables=[]
    for i in range(args.solDim*args.card):
        variables.append(MODEL.addVar(vtype=gurobipy.GRB.BINARY, name='x'+'i'))
    
    MODEL.update()

    
    MODEL.setObjective(np.array(variables).dot(theta), sense=gurobipy.GRB.MAXIMIZE)

    
    cnt=0
    for i in range(args.solDim):
        if len(legalList)>0:
            for j in legalList[i]:
                tmp=0
                for line in range(args.card): 
                    tmp=tmp-variables[line*args.solDim+i]-variables[line*args.solDim+j] 
                    
                MODEL.addConstr(tmp>= -1, name=str(cnt))
                cnt+=1
    MODEL.addConstr(sum(variables) == args.card, name=str(cnt))
    f=time.time()
    
    MODEL.optimize()
    return np.array(MODEL.x)
def solver_quad(Q,MODEL):

    import gurobipy
    import time
    s=time.time()
    
    variables=[]
    for i in range(args.solDim*args.card):
        variables.append(MODEL.addVar(vtype=gurobipy.GRB.BINARY, name='x'+'i'))
    
    MODEL.update()

    
    MODEL.setObjective(np.array(variables).dot(Q).dot(np.array(variables)), sense=gurobipy.GRB.MAXIMIZE)

    
    cnt=0
    for i in range(args.solDim):
        if len(legalList)>0:
            for j in legalList[i]:
                tmp=0
                for line in range(args.card): 
                    tmp=tmp-variables[line*args.solDim+i]-variables[line*args.solDim+j] 
                    
                MODEL.addConstr(tmp>= -1, name=str(cnt))
                cnt+=1
    MODEL.addConstr(sum(variables) == args.card, name=str(cnt))
    f=time.time()
    
    
    MODEL.optimize()
    return np.array(MODEL.x)
def solver_mixed(Q,a,MODEL):
    import gurobipy
    import time
    s=time.time()
    
    variables=[]
    for i in range(args.solDim*args.card):
        variables.append(MODEL.addVar(vtype=gurobipy.GRB.BINARY, name='x'+'i'))
    
    MODEL.update()

    
    MODEL.setObjective(np.array(variables).dot(Q).dot(np.array(variables))+(np.array(variables).dot(a))*(np.array(variables).dot(a)), sense=gurobipy.GRB.MAXIMIZE)

    
    cnt=0
    for i in range(args.solDim):
        if len(legalList)>0:
            for j in legalList[i]:
                tmp=0
                for line in range(args.card): 
                    tmp=tmp-variables[line*args.solDim+i]-variables[line*args.solDim+j] 
                    
                MODEL.addConstr(tmp>= -1, name=str(cnt))
                cnt+=1
    MODEL.addConstr(sum(variables) == args.card, name=str(cnt))
    f=time.time()
    
    
    MODEL.optimize()
    return np.array(MODEL.x)

In [4]:
def onehot_to_rank(onehot):
    if type(onehot)!=np.ndarray:
        onehot=onehot.detach().numpy()
    onehot=onehot.reshape([args.card,args.solDim])
    vec=[]
    for i in range(args.card):
        if len(np.argwhere(onehot[i]>0.1))!=0:
            vec.append(np.argwhere(onehot[i]>0.1)[0])
    return np.array(vec)
def posdata_to_onehot(rank):
    matrix=np.zeros([args.card,args.solDim])
    for i in range(args.card):
        matrix[i][rank[i]]=1  
    return matrix.reshape(args.card*args.solDim)

In [5]:
def klBern(x, y):
    r""" Kullback-Leibler divergence for Bernoulli distributions. https://en.wikipedia.org/wiki/Bernoulli_distribution#Kullback.E2.80.93Leibler_divergence

    .. math:: \mathrm{KL}(\mathcal{B}(x), \mathcal{B}(y)) = x \log(\frac{x}{y}) + (1-x) \log(\frac{1-x}{1-y})."""
    eps=1e-5
    x = min(max(x, eps), 1 - eps)
    y = min(max(y, eps), 1 - eps)
    return x * np.log(x / y) + (1 - x) * np.log((1 - x) / (1 - y))

In [ ]:
import scipy.optimize as opt
import scipy
from numpy import *
for rnd in range(3,10):
    rewardlist=[]
    constraintlist=[]
    T=[1]*len(wlist)
    w=[0]*len(wlist)
    U=[0]*len(wlist)
    cnt=1
    def UCB(x):
        left=wlist[x]
        right=1
        thre=(np.log(cnt)+3*np.log(np.log(cnt)))/T[x]
        res=(left+right)/2
        if klBern(wlist[x],res)<thre and abs(klBern(wlist[x],res)-thre)>1e-3:
            left=res
            res=(left+right)/2
        else:
            if abs(klBern(wlist[x],res)-thre)<1e-3:
                return res
            else:
                right=res
                res=(left+right)/2
        return res
    while cnt<=5001:
        U=np.array([UCB(i) for i in range(len(wlist))])
        action=np.zeros(args.card).astype(int)
        for i in range(args.card):
            tmp=np.argwhere(U.argsort()==(len(wlist)-1-i))[0][0]
            if type(tmp)!=np.int64:
                action[i]=int(tmp[0])
            else:
                action[i]=int(tmp)
        C=hwithoutconstraints(action)
        for i in range(args.solDim):
            if C[i]==1:
                T[i]+=1
                w[i]=((T[i]-1)*w[i]+1)/T[i]
        rewardlist.append(sum(C))
        constraintlist.append(posdata_to_onehot(action))
        cnt+=1
        if cnt%100==0:
            print(cnt)
            np.save('cascadingUCBreward'+str(rnd)+'.npy',rewardlist)
            np.save('cascadingUCBconstraint'+str(rnd)+'.npy',constraintlist)

In [ ]:
#https://arxiv.org/pdf/1502.02763.pdf

In [ ]:
plt.plot(np.array(list(range(50,100)))/100,[klBern(0.4,i) for i in np.array(list(range(50,100)))/100])